In [12]:
import numpy as np
from  user import *
from  item import *
from user_based_Recommendor import *
from item_based_Recommendor import *

In [13]:
#there are 943 users and 1682 items
user_item_matrix=np.full((943+1,1682+1),-1) #each row represents a user and each col represents a item, user_item_matrix[i][j]=user i's rating of item j
file = open('ml-100k/u.data', 'r')
Lines = file.readlines()
for line in Lines:
    splits=line.strip().split('\t')
    user_id=int(splits[0])
    item_id=int(splits[1])
    rating=int(splits[2])

    user_item_matrix[user_id][item_id]=rating

In [14]:
#there are 943 users and 1682 items
item_user_matrix=np.full((1682+1,943+1),-1) #each row represents a user and each col represents a item, user_item_matrix[i][j]=user i's rating of item j
file = open('ml-100k/u.data', 'r')
Lines = file.readlines()
for line in Lines:
    splits=line.strip().split('\t')
    user_id=int(splits[0])
    item_id=int(splits[1])
    rating=int(splits[2])

    item_user_matrix[item_id][user_id]=rating

In [15]:
user_dict={}
file = open('ml-100k/u.user', 'r')
Lines = file.readlines()
for line in Lines:
    splits=line.strip().split('|')
    user_id=splits[0]

    new_user=user(user_id,splits[1],splits[2],splits[3],splits[4])#id,age,gender,occupation,zip_code
    user_dict[user_id]=new_user


In [16]:
print(user_dict['1'].printUserInfor())

User id =1, age=24, gender=M, occupation=technician, zip code=85711
None


In [17]:
item_dict={}
file = open('ml-100k/u.item', 'r',encoding="ISO-8859-1")
Lines = file.readlines()
for line in Lines:
    splits=line.strip().split('|')#1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0
    movie_id=splits[0]
    new_movie=movie(item_id,splits[1],splits[2],splits[3],splits[4],getTypes(line[-38:]))
    item_dict[movie_id]=new_movie

In [18]:
print(item_dict['1'].printMovieInfor())

movie id=203
movie title=Toy Story (1995)
release date=01-Jan-1995
video release date=
IMDB URL=http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)
type=Animation,Children's,Comedy
None


In [19]:
#user_based_Model_Test(user_item_matrix,3)

In [20]:
neighbor_size=[3,6,9,12,15]
for n in range(neighbor_size):
    print("n="+str(n))
    user_based_Model_Test(user_item_matrix,n)

In [21]:
print(item_user_matrix.shape)

(1683, 944)


In [22]:
neighbor_size=[3,6,9,12,15]
for n in neighbor_size:
    print("n="+str(n))
    item_based_Model_Test(item_user_matrix,n)

n=3
  0%|          | 0/1682 [00:41<?, ?it/s]


KeyboardInterrupt: 

SVD

In [23]:
from surprise import SVD, NMF, accuracy,SVDpp
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate, train_test_split
import pandas as pd

In [24]:
def load_movies_dataset() -> pd.DataFrame:
    movie_data_columns = [
    'movie_id', 'title', 'release_date', 'video_release_date', 'url',
    'unknown', 'Action', 'Adventure', 'Animation', "Children's",
    'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
    'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller',
    'War', 'Western'
    ]

    movie_data = pd.read_csv(
        'datasets/ml-100k/u.item', 
        sep = '|', 
        encoding = "ISO-8859-1", 
        header = None, 
        names = movie_data_columns,
        index_col = 'movie_id'
    )
    movie_data['release_date'] = pd.to_datetime(movie_data['release_date'])
    return movie_data

def load_ratings() -> pd.DataFrame:
    ratings_data = pd.read_csv(
        'ml-100k/u.data',
        sep = '\t',
        encoding = "ISO-8859-1",
        header = None,
        names=['user_id', 'movie_id', 'rating', 'timestamp']
    )
    
    return ratings_data[['user_id', 'movie_id', 'rating']]

In [25]:
ratings_data = load_ratings()
ratings_data.head(10)

,user_id,movie_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1
5,298,474,4
6,115,265,2
7,253,465,5
8,305,451,3
9,6,86,3


In [40]:
min_rating, max_rating = ratings_data['rating'].min(), ratings_data['rating'].max()

reader = Reader(rating_scale=(min_rating, max_rating))
data = Dataset.load_from_df(ratings_data, reader)
trainset, testset = train_test_split(data, test_size=.25)

n_factors=[20,40,60,80,100]
for f in n_factors:
    print("f="+str(f))
    model = SVD(n_factors=f, biased=False)
    model.fit(trainset)

    # Let's calculate the RMSE
    predictions = model.test(testset)
    accuracy.rmse(predictions)

f=20
RMSE: 0.9383
f=40
RMSE: 0.9405
f=60
RMSE: 0.9410
f=80
RMSE: 0.9449
f=100
RMSE: 0.9474


In [27]:
print(predictions[0])

user: 916        item: 642        r_ui = 3.00   est = 3.94   {'was_impossible': False}


In [28]:
n_factors=[20,40,60,80,100]
for f in n_factors:
    model = SVDpp(n_factors=100)
    model.fit(trainset)

    predictions = model.test(testset)
    accuracy.rmse(predictions)

RMSE: 0.9287


0.9286967871115341